# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Choose the topic ✅

In [ ]:
import sys
sys.path.append("../src")

from chat.chatbot import Socrates, Eris
# from audio.speak import stream_message, stream_first_message

from chat.test_chatbot import TestEris, TestSocrates
from streaming.stream import init_conversation, stream_with_typing
# from audio.test_speak import speak_message

import gradio as gr
import random

In [ ]:
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
# !winget install ffmpeg        # Required with pydub

## Debate: Claude VS GPT

### Main Function

In [37]:
def adversarial_chat(n_rounds, topic, delay=0.005):
    """
    Enhanced debate with proper streaming and typing effects.
    """
    socrates = TestSocrates()
    eris = TestEris()
    history = []

    # Start the heated debate - use simple streaming for the opening message
    eris_options = [
        f"Ready to get schooled about '{topic}'?, old man?", 
        f"Today's topic is: '{topic}'. This should be easy—your logic is about as sharp as a wet napkin.",
        f"Hope you're prepared to lose spectacularly, Socrates. Let us talk about: '{topic}'",
        f"Hey Socrates. Wanna argue about: '{topic}'? It will be fun to see you cry once you have lost.",
    ]
    eris_reply = random.choice(eris_options)
    
    # Stream the first message
    for update in stream_with_typing(history=history, message=eris_reply, is_socrates=False, delay=delay):
        # history = update  # Update the history reference
        yield update

    # Continue with alternating turns using real API streaming
    for i in range(n_rounds-1):
        if i % 2 == 0:
            # Socrates takes even turns
            socrates.history = eris.history.copy()  # Sync conversation history
            socrates_reply = socrates.stream(eris_reply)
            
            for update in stream_with_typing(history=history, message=socrates_reply, is_socrates=True, delay=delay):
                # history = update  # Update the history reference
                yield update
            soc_reply = history[-1]["content"]
            
        else:
            # Eris takes odd turns
            eris.history = socrates.history.copy()  # Sync conversation history
            eris_reply = eris.stream(soc_reply)

            for update in stream_with_typing(history=history, message=eris_reply, is_socrates=False, delay=delay):
                # history = update  # Update the history reference
                yield update
            eris_reply = history[-1]["content"]



### GUI

In [38]:
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0, minimum=1)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate", interactive=True) #False)  # Initially disabled

    # chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=500)
    chatbox = gr.Chatbot(
        label="Let the battle begin!",
        type="messages",
        value=[],
        height=500
    )

    def reset_chat():
        return []

    def validate_inputs(topic, n_rounds):
        valid_topic = bool(topic.strip())
        valid_rounds = isinstance(n_rounds, (int, float)) and n_rounds > 0
        return gr.update(interactive=valid_topic and valid_rounds)

    # Hook to clear previous chat
    start_btn.click(fn=reset_chat, outputs=[chatbox])

    # Launch the debate
    start_btn.click(fn=adversarial_chat,
                    inputs=[rounds, topic_input],
                    outputs=[chatbox])

    # Enable/disable start button based on both inputs
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)


demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.
